This notebook is for training softmax regression, the code resides in **softmax_digits.py**. Algorithm is implemented in **softmax_example.py and nnbase.py**. nnbase can be adapted for variety of neural network types. 
Data is truncated mnist provided by Taras

In [2]:
#%load_ext autotime
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sns.set(color_codes=True)
plt.rcParams['savefig.dpi'] = 100

from IPython.display import display

import softmax_digits as softdg
from softmax_example import SoftmaxRegression

In [4]:
# read and preprocess data, see softmax_digits.py
(X_train, y_train, X_dev, y_dev, X_test, y_test, X_fulltest) = softdg.get_data(num_train=4800, 
        num_dev=100, num_test=100)

In [5]:
# Here we try different batch size strategies
import random
nepoch = 5
sz = len(y_train)
print 'Training data size is', sz
N = nepoch * len(y_train)
k = 8 # minibatch size

random.seed(10)
def epoch_sch():
    for i in xrange(nepoch):
        for j in xrange(sz):
            yield j

def rand_sch():
    for i in xrange(N):
        yield random.randint(0, sz)

def rand_minibatch(n, k):
    # minibatch with size k
    for _ in xrange (n / k):
        ret = [np.random.randint(0, sz) for _ in xrange(k)] 
        yield ret
def fullbatch():
    for i in range(500):
        ret = range(sz)
        yield ret


In [44]:
sr = SoftmaxRegression(wv=np.zeros_like(X_train), dims=(400, 10), alpha=0.0001, reg=0.01)
curve = sr.train_sgd(X_train, y_train, fullbatch(), costevery=100, devX=X_dev, devy=y_dev)
counts, costs, costdevs  = zip(*curve)

In [45]:
# plot train and dev errors
plt.figure(figsize=(6,4))

plt.plot(5*np.array(counts), costs, color='b', marker='o', linestyle='-', label=r"train_loss")
plt.plot(5*np.array(counts), costdevs, color='g', marker='o', linestyle='-', label=r"validation_loss")

plt.title(r"Learning Curve ($\lambda=0.001$, costevery=5000)")
plt.xlabel("SGD Iterations"); plt.ylabel(r"Average $J(\theta)$"); 
plt.ylim(ymin=0, ymax=max(1.1*max(costs),3*min(costs)));
plt.legend()
#plt.show()

In [46]:
accuracy_test = np.count_nonzero(sr.predict(X_test) == y_test) / float(len(y_test))
print 'accuracy on test', accuracy_test
accuracy_test = np.count_nonzero(sr.predict(X_train) == y_train) / float(len(y_train))
print 'accuracy on test', accuracy_test